In [1]:
import time

import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--dns-prefetch-disable')
#driver = Chrome(chrome_options=options)

empty_dww = pd.DataFrame()
empty_district_info = pd.DataFrame()
empty_audit_info = pd.DataFrame()
driver = webdriver.Chrome(chrome_options=chrome_options)  # Optional argument, if not specified will search path.
base_page = 'http://www14.tceq.texas.gov/iwud/dist/index.cfm?fuseaction=ListDistricts&COMMAND=LIST&compress=N&StartName=&ID=&RegionCode=&DistTypeCode=&CreationTypeCode=&DistFunctionTypeCode=&CountyCode=&FinancialStatus=&ActivityStatus=&ListStart='
driver.get(base_page)
#CountyCode

houston_counties = [79,101,170,84]

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
#
# select by value 
#select.select_by_value('1')
#wait = WebDriverWait(driver, 10)
select = Select(driver.find_element_by_css_selector('#CountyCode'))
wd_info_call = '.iwuddata:nth-child(2)'
#wd_info_header = 'tr:nth-child(5) .iwudheading , .iwudheading:nth-child(1)'
wd_info_header = '.iwudheading:nth-child(1)'
#for county in houston_counties:
count = 0
select_county = Select(driver.find_element_by_css_selector('#CountyCode'))
select_county.select_by_value(str(36))
#select_status = Select(driver.find_element_by_css_selector('#ActivityStatus'))
#select_status.select_by_value(str('A'))
driver.find_elements_by_css_selector('input')[1].click()
css_find_wds = "table+ table .iwud a"
css_find_number = '.iwud+ .iwud:nth-child(3)'
css_find_status = '.iwud~ .iwud+ .iwud'
css_to_click_more = "table+ table tr:nth-child(1) a:nth-child(1) img"
audit_vars_css = 'td.iwud:nth-child(1)'
audit_vals_css = '.iwud+ td'
get_doc_id = 'table:nth-child(6) tr:nth-child(1) .iwuddata'

System = [];Status = []; ID = []
#### Grab basic district info
count = 0

for system in range(2000):
    print('system' + str(system))
    temp_wd = driver.find_elements_by_css_selector(css_find_wds)
    temp_status = driver.find_elements_by_css_selector(css_find_status)
    temp_number = driver.find_elements_by_css_selector(css_find_number)
    System.append(temp_wd[count].text)
    Status.append(temp_status[count].text)
    ID.append(temp_number[count].text)
    water_systems = driver.find_elements_by_css_selector(css_find_wds)
    #for systems in range(len(water_systems)):
    temp_name = driver.find_elements_by_css_selector(css_find_wds)[count].text
    print(temp_name)
    temp_id = driver.find_elements_by_css_selector(css_find_number)[count].text
    driver.find_elements_by_css_selector(css_find_wds)[count].click()
    info_vars = []
    info = []
    for el in driver.find_elements_by_css_selector(wd_info_call):
        info.append(el.text)
    for el in driver.find_elements_by_css_selector(wd_info_header):
        info_vars.append(el.text)
    i = 0
    for v in range(len(info_vars)):
        if info_vars[v] == '':
            i = i + 1
            info_vars[v] = str(i)
            #print(info_vars);print(info)
    #print(info_vars)
    #print(info)
    temp = pd.DataFrame(info).T
    temp.columns = [info_vars]
    temp['NAME'] = temp_name
    temp['WDD_ID'] = temp_id
    #for dup in range(sum(temp.columns.duplicated())):
    while sum(temp.columns.duplicated())>0:
        temp = temp.drop(temp.columns[temp.columns.duplicated()][0],axis=1)
        #temp = temp.drop('Address:', axis=1)
        #temp = temp.columns.drop_duplicates()
        #print(temp_info)
        #if empty_dww.shape[0] !=0 & any(empty_dww['Name'] == temp_name):
        #    driver.back()
        #    continue  join(temp_info,how='outer')
        #print(temp.columns)
    empty_district_info = pd.concat([empty_district_info,temp], axis=0, join='outer', ignore_index=True)
    empty_district_info.to_csv('../Input/tceq_audits/chambers_system_info.csv')
        #empty_district_info = empty_district_info.append(temp,how='outer')
        #empty_district_info =  = empty_district_info.append(temp)
        #grouped = empty_district_info.groupby(level=0)
        #empty_district_info = grouped.last()
    time.sleep(1)    
    driver.find_element_by_css_selector('.iwud a+ a img').click()
        #doc_list = driver.find_elements_by_css_selector('.iwudcolor+ tr td')
    audit_links = driver.find_elements_by_partial_link_text('AUDIT')
    if (len(audit_links)==0):
        driver.back()
    if (len(audit_links)!=0):
        for audit in range(len(audit_links)):
            audit_links = driver.find_elements_by_partial_link_text('AUDIT')
            doc_link = audit_links[audit].get_attribute('href')
            audit_links[audit].click()   
            avars = []; avals = []
            doc_id = driver.find_element_by_css_selector(get_doc_id).text
            #print('docid')
            #print(doc_id)
            for var in driver.find_elements_by_css_selector(audit_vars_css):
                avars.append(var.text)
            for var in driver.find_elements_by_css_selector(audit_vals_css):
                avals.append(var.text)
            temp_audit = pd.DataFrame(avals).T
            temp_audit.columns = avars
            temp_audit['DOC_ID'] = doc_id
            temp_audit['SYSTEM_NAME'] = temp_name
            temp_audit['SYSTEM_ID'] = temp_id
            temp_audit['DOC_LINK'] = doc_link
                #temp_audit,index=avars,columns = [doc_id])
                #temp_audit = temp_audit.append(pd.DataFrame([id],index=['System ID'],columns=[doc_id]))
                #temp_audit = temp_audit.append(pd.DataFrame([doc_link],index=['Doc_Link'],columns=[doc_id]))
            empty_audit_info = pd.concat([empty_audit_info,temp_audit],axis=0, join='outer', ignore_index=True)
            empty_audit_info.to_csv('../Input/tceq_audits/chambers_audits.csv')
            time.sleep(1) # delays for 5 seconds
            driver.back()
        time.sleep(1)
        driver.back()
    time.sleep(1)
    driver.back()
    if len(temp_wd)<49 & count == len(temp_wd):
        break
    if count == 49:
        driver.find_element_by_css_selector(css_to_click_more).click()
        count = 0
        continue
    count = count + 1
system_list_df = pd.DataFrame({'Name': System, 'Status': Status,'ID': ID,'County': 'CHAMBERS'})
#empty_dww = empty_dww.append(system_list_df)       
#harris_dww = system_list_df
system_list_df.to_csv('../Input/tceq_audits/chambers_base.csv')
#harris_audits = empty_audit_info
#harris_systems = empty_district_info


#while len(driver.find_element_by_css_selector(css_to_click_more))>0:
#    driver.find_element_by_css_selector(css_to_click_more).click()



system0
CEDAR BAYOU MUD
system1
CEDAR BAYOU NAVIGATION DISTRICT
system2
CHAMBERS COUNTY DRAINAGE DISTRICT 1
system3
CHAMBERS COUNTY DRAINAGE DISTRICT 2
system4
CHAMBERS COUNTY IMPROVEMENT DISTRICT 1
system5
CHAMBERS COUNTY IMPROVEMENT DISTRICT 2
system6
CHAMBERS COUNTY IMPROVEMENT DISTRICT 3
system7
CHAMBERS COUNTY MUD 1
system8
CHAMBERS COUNTY WCID 1
system9
CHAMBERS-LIBERTY COUNTIES NAVIGATION DISTRICT
system10
COASTAL WATER AUTHORITY
system11
GULF COAST WASTE DISPOSAL AUTHORITY
system12
LOWER NECHES VALLEY AUTHORITY
system13
OLD RIVER COUNTRY MUD
system14
SOUTHEAST TEXAS AGRICULTURAL DEVELOPMENT DISTRICT
system15
TRINITY BAY CONSERVATION DISTRICT
system16
TRINITY RIVER AUTHORITY OF TEXAS
system17


IndexError: list index out of range

In [2]:
system_list_df = pd.DataFrame({'Name': System, 'Status': Status,'ID': ID,'County': 'CHAMBERS'})
#empty_dww = empty_dww.append(system_list_df)       
#harris_dww = system_list_df
system_list_df.to_csv('../Input/tceq_audits/chambers_base.csv')